## l'esigenza è quella di valorizzare una nuova colonna del dataframe con valori definiti in un dizionario le cui chiavi possono o meno corrispondere ai valori del dataframe index (timestamp)


## TBD: test prestazionali con perfplot (modificare le dim. del dict rispetto al df)
## TBD: test np.fromiter (perfplot_map_fx_numpy_arr.ipynb)
https://stackoverflow.com/questions/35215161/most-efficient-way-to-map-function-over-numpy-array/46470401#46470401
https://stackoverflow.com/questions/8486294/how-to-add-an-extra-column-to-a-numpy-array/40218298#40218298
https://stackoverflow.com/questions/9236926/concatenating-two-one-dimensional-numpy-arrays

In [7]:
import datetime
import pandas as pd
import numpy as np

todays_date = datetime.datetime.now().date()
delta_days  = 100
date_range  = pd.date_range(todays_date - datetime.timedelta(delta_days), periods=delta_days+1, freq='D')
df          = pd.DataFrame(index=date_range, columns=['x','y']).fillna(0)

In [8]:
from random import choice, randint

kind_of_day   = ["special day", "work day"]
my_choice     = choice(kind_of_day)

_dict_pd_date_range  = {dt : choice(kind_of_day) for dt in date_range}
_vectorized_dict     = {dt : choice(kind_of_day) for dt in df.index.values}

In [9]:
# rimuovo un elem. qualsiasi x il test funzionale
#_vectorized_dict
del _vectorized_dict[np.datetime64('2018-10-22T00:00:00.000000000')]

In [10]:
# rimuovo un elem. qualsiasi x il test funzionale
#_dict_pd_date_range
del _dict_pd_date_range[pd.Timestamp('2018-10-11 00:00:00', freq='D')]

In [11]:
def check_day_pd_date_range (row):
    '''
    45.8 ms
    '''
    key = pd.to_datetime(row.name)
    if key in _dict_pd_date_range:
        return _dict_pd_date_range[key]
    else:
        return '-'
        
    
def vectorized_check_day (indexes):
    '''
    3.72 ms (~ 12x)
    '''
    return [_vectorized_dict[index] if index in _vectorized_dict else None for index in indexes]
 
    
def np_column_stack (indexes):
    '''
    np_column_stack NON migliora le performance di vectorized_check_day (indexes)
    
    '''
    target = np.full(indexes.size, None)   # default filled column
    
    #dst = np.dstack((indexes,target)).reshape(indexes.size, 2) ## OK...
    dst = np.column_stack( (indexes, target) ) ### BETTER
    #dst = np.column_stack((indexes, np.full(indexes.size, None)))
    
    for k,v in _vectorized_dict.items():
        dst[np.where(dst[:,0] == k), 1] = v
    
    return dst[:, 1] # return column 1


def np_concatenate (indexes):
    '''
    equivalente alla np_column_stack
    '''
    
    target = np.full(indexes.size, None)
    dst    = np.concatenate([indexes[:, None], target[:, None]], axis=1) # perfplot example
    
    for k,v in _vectorized_dict.items():
        dst[np.where(dst[:,0] == k), 1] = v
    
    return dst[:, 1]
    
'''    
def f1(a, d):
    b = copy(a)
    for k, v in d.iteritems():
        b[a==k] = v
    return b
'''

 


'    \ndef f1(a, d):\n    b = copy(a)\n    for k, v in d.iteritems():\n        b[a==k] = v\n    return b\n'

In [6]:
len(_vectorized_dict)

1000

In [7]:
%%timeit 
df['special_day'] = vectorized_check_day(df.index.values)

100 loops, best of 3: 3.63 ms per loop


In [8]:
%%timeit
df['special_day'] = df.apply(lambda row: check_day_pd_date_range(row), axis=1)

10 loops, best of 3: 46.5 ms per loop


In [16]:
%%timeit
df['special_day'] = np_column_stack(df.index.values)

10 loops, best of 3: 21.8 ms per loop


In [15]:
%%timeit
df['special_day'] = np_concatenate(df.index.values)

10 loops, best of 3: 21.8 ms per loop


In [39]:
df

,x,y,special_day
2018-10-12,0,0,special day
2018-10-13,0,0,special day
2018-10-14,0,0,special day
2018-10-15,0,0,special day
2018-10-16,0,0,work day
2018-10-17,0,0,work day
2018-10-18,0,0,special day
2018-10-19,0,0,work day
2018-10-20,0,0,work day
2018-10-21,0,0,special day


In [10]:
_x = vectorized_check_day(df.index.values)

In [11]:
x = pd.DataFrame.from_dict(_vectorized_dict, orient='index').sort_index()
x

,0
2018-10-10,work day
2018-10-11,work day
2018-10-12,work day
2018-10-13,special day
2018-10-14,work day
2018-10-15,work day
2018-10-16,work day
2018-10-17,special day
2018-10-18,work day
2018-10-19,work day


In [ ]:
# https://stackoverflow.com/questions/46868855/numpy-replacing-values-in-a-2d-array-efficiently-using-a-dictionary-as-a-map

a = np.array([[  3,   0,   2,  -1],
              [  1, 255,   1,   2],
              [  0,   3,   2,   2]])

d = {0: 1, 1: 2, 2: 3, 3: 4, -1: 0, 255: 0}

In [ ]:
# https://stackoverflow.com/questions/3403973/fast-replacement-of-values-in-a-numpy-array
from numpy import copy, random, arange

random.seed(0)
data = random.randint(30, size=10**5)

#d = {4: 0, 9: 5, 14: 10, 19: 15, 20: 0, 21: 1, 22: 2, 23: 3, 24: 0}
# d = _vectorized_dict
dk = d.keys()
dv = d.values()

def f1(a, d):
    b = copy(a)
    for k, v in d.iteritems():
        b[a==k] = v
    return b

a = copy(data)
res = f2(a, d)

assert (f1(data, d) == res).all()
assert (f3(data, dk, dv) == res).all()

In [ ]:
# https://stackoverflow.com/questions/3403973/fast-replacement-of-values-in-a-numpy-array

import timeit
import numpy as np

array = 2 * np.round(np.random.uniform(0,10000,300000)).astype(int)
from_values = np.unique(array) # pair values from 0 to 2000
to_values = np.arange(from_values.size) # all values from 0 to 1000
d = dict(zip(from_values, to_values))

def method_for_loop():
    out = array.copy()
    for from_value, to_value in zip(from_values, to_values) :
        out[out == from_value] = to_value
    print('Check method_for_loop :', np.all(out == array/2)) # Just checking
print('Time method_for_loop :', timeit.timeit(method_for_loop, number = 1))

def method_list_comprehension():
    out = [d[i] for i in array]
    print('Check method_list_comprehension :', np.all(out == array/2)) # Just checking
print('Time method_list_comprehension :', timeit.timeit(method_list_comprehension, number = 1))

def method_bruteforce():
    idx = np.nonzero(from_values == array[:,None])[1]
    out = to_values[idx]
    print('Check method_bruteforce :', np.all(out == array/2)) # Just checking
print('Time method_bruteforce :', timeit.timeit(method_bruteforce, number = 1))

def method_searchsort():
    sort_idx = np.argsort(from_values)
    idx = np.searchsorted(from_values,array,sorter = sort_idx)
    out = to_values[sort_idx][idx]
    print('Check method_searchsort :', np.all(out == array/2)) # Just checking
print('Time method_searchsort :', timeit.timeit(method_searchsort, number = 1))

In [ ]:
# https://stackoverflow.com/questions/46868855/numpy-replacing-values-in-a-2d-array-efficiently-using-a-dictionary-as-a-map
# Function to be vectorized
def map_func(val, dictionary):
    return dictionary[val] if val in dictionary else val 

# Vectorize map_func
vfunc  = np.vectorize(map_func)

# Run
print(vfunc(a, d))


from timeit import Timer
t = Timer('vfunc(a, d)', 'from __main__ import a, d, vfunc')
print(t.timeit(number=1000))

In [ ]:
# https://stackoverflow.com/questions/46868855/numpy-replacing-values-in-a-2d-array-efficiently-using-a-dictionary-as-a-map

a = np.array([[  3,   0,   2,  -1],
              [  1, 255,   1,   2],
              [  0,   3,   2,   2]])

d = {0: 1, 1: 2, 2: 3, 3: 4, -1: 0, 255: 0}

#indexer = np.array([d.get(i, -1) for i in range(a.min(), a.max() + 1)])
indexer = np.array([d.get(i, i) for i in range(a.min(), a.max() + 1)])

In [12]: indexer[(a - a.min())]
Out[12]:
array([[4, 1, 3, 0],
       [2, 0, 2, 3],
       [1, 4, 3, 3]])


In [ ]:
# https://stackoverflow.com/questions/46868855/numpy-replacing-values-in-a-2d-array-efficiently-using-a-dictionary-as-a-map
import numpy as np
b = np.copy(a)
for old, new in d.items():
    b[a == old] = new

In [ ]:
# https://stackoverflow.com/questions/46868855/numpy-replacing-values-in-a-2d-array-efficiently-using-a-dictionary-as-a-map
def getval_array(d):
    v = np.array(list(d.values()))
    k = np.array(list(d.keys()))
    maxv = k.max()
    minv = k.min()
    n = maxv - minv + 1
    val = np.empty(n,dtype=v.dtype)
    val[k] = v
    return val

val_arr = getval_array(d)